# Bulding model for schema

![database schema](../Material/ER-1.png)

### SQL Query for creating a table

The query for the creation of the table can be found in this "Database.sql"

[Schema](../Database.sql)

## Building entites data

In [1]:
from csvRW import read_csv, create_csv, read_csv_without_header, append_row_to_csv

#### Source Schema
```
Source(_source_id_, source_name, source_website)
```

In [2]:
raw_data = read_csv("../covid-19-data/locations.csv")
result = list(set([(data['source_name'],data['source_website']) for data in raw_data]))
result.append(['',''])
create_csv("../model/Source.csv",result)

File is successfully read ../covid-19-data/locations.csv
File is successfully created at ../model/Source.csv


#### Country Schema 

Read all country iso
```
Country(_iso_code_*,location, source_id*, last_observation_date)
```

In [3]:
raw_data = read_csv("../covid-19-data/iso.csv")
resultLocation ={}
for data in raw_data:
    resultLocation[data['iso_code']] = data["location"]
raw_data = read_csv("../covid-19-data/locations.csv")
source = read_csv_without_header("../model/Source.csv")

result = []
for data in raw_data:
    if [data['source_name'],data['source_website']] in source:
        result.append([data['iso_code'],data['location'],source.index([data['source_name'],data['source_website']])+1, data['last_observation_date']])
        del resultLocation[data['iso_code']]
    else:
        print("Missed country",data['iso_code'])

for iso_code, location in resultLocation.items():
    result.append([iso_code,location,108,''])

create_csv("../model/Country.csv",result)

File is successfully read ../covid-19-data/iso.csv
File is successfully read ../covid-19-data/locations.csv
File is successfully created at ../model/Country.csv


#### Age_Group Schema
```
Age_group(_age_group_)
```

In [4]:
raw_data = read_csv("../covid-19-data/vaccinations-by-age-group.csv")

result = set([data['age_group']for data in raw_data])
result = [[data]for data in result]
create_csv("../model/Age_group.csv",result)

File is successfully read ../covid-19-data/vaccinations-by-age-group.csv
File is successfully created at ../model/Age_group.csv


#### Vaccine Schema
```
Vaccine(_vaccine_)
```

In [5]:
raw_data_loc = read_csv("../covid-19-data/locations.csv")
raw_data_manu = read_csv("../covid-19-data/vaccinations-by-manufacturer.csv")

result = []
for data in raw_data_loc:
    result.append(data['vaccines'].split(","))

result = [x.strip() for xs in result for x in xs]

for data in raw_data_manu:
    result.append(data['vaccine'].strip())

result = [[val]for val in list(set(result))]

create_csv("../model/Vaccine.csv",result)

File is successfully read ../covid-19-data/locations.csv
File is successfully read ../covid-19-data/vaccinations-by-manufacturer.csv
File is successfully created at ../model/Vaccine.csv


#### Population_Country schema
```
Population_Country(_iso_code_*, year, population)
```

In [6]:
raw_data_loc = read_csv_without_header("../model/Country.csv")
raw_data_loc = [data[0] for data in raw_data_loc]

raw_data = read_csv("../covid-19-data/population_latest.csv")

result = [[data['iso_code'],data['year'],data['population']] for data in raw_data if data['iso_code'] in raw_data_loc]

create_csv("../model/Population_Country.csv",result)

File is successfully read ../covid-19-data/population_latest.csv
File is successfully created at ../model/Population_Country.csv


#### Country_Vaccine Schema
```
Country_Vaccine(_iso_code_*,_vaccine_*)
```


In [11]:
raw_data_iso = read_csv_without_header("../model/Country.csv")
raw_data_iso = [data[0] for data in raw_data_iso]

raw_data_vac = read_csv_without_header("../model/Vaccine.csv")
raw_data_vac = [data[0] for data in raw_data_vac]

raw_data_loc = read_csv("../covid-19-data/locations.csv")

result = []
for data in raw_data_loc:
    vaccineList = [val.strip() for val in data['vaccines'].split(",")]

    if data['iso_code'] in raw_data_iso:
        for vaccine in vaccineList:
            if vaccine in raw_data_vac:
                result.append((data['iso_code'],vaccine))
            else:
                print("No entered vaccine table ",vaccine)
    else:
        print("No entered country table ",data['iso_code'])

create_csv("../model/Country_Vaccine.csv",set(result))


File is successfully read ../covid-19-data/locations.csv
File is successfully created at ../model/Country_Vaccine.csv


#### Manufacture_date Schema

```Manufacture_date(_iso_code_*, _date_, _vaccine_*, total_vaccinations)```

In [9]:
raw_data_iso = read_csv_without_header("../model/Country.csv")
iso_map = {}

for data in raw_data_iso:
    iso_map[data[1]] = data[0]
    
result = []

for data in raw_data_manu:
    if data['location'] in iso_map.keys():
        if data['vaccine'] in raw_data_vac:
           result.append([iso_map[data['location']],data['date'],data['vaccine'], data['total_vaccinations']])
        else:
            print(data['vaccine']," is not in vaccine table")
            break
    else:
        print(data['location']," is not in country table")  

create_csv("../model/Manufacture_date.csv",result)

File is successfully created at ../model/Manufacture_date.csv


#### Vaccination Schema

```Vaccination(_iso_code_*, _date_, total_vaccinations, people_vaccinated, people_fully_vaccinated, total_booster, daily_vaccination_raw, daily_vaccination, daily_people_vaccinated)```

In [ ]:
iso_list = iso_map.values()
raw_data_vac = read_csv("../covid-19-data/vaccinations.csv")

result = []
for data in raw_data_vac:
    if data['iso_code'] in iso_list:
        pass
    else:
        print("Missing iso code in country table",data['iso_code'])
        append_row_to_csv("../model/Country.csv",[data['iso_code'],data['location'],'',''])
    
    result.append([data['iso_code'],data['date'],data['total_vaccinations'],data['people_vaccinated'],data['people_fully_vaccinated'],data['total_boosters'],data['daily_vaccinations_raw'],data['daily_vaccinations'], data['daily_people_vaccinated']])

# Reload the data of country table

create_csv("../model/Vaccination.csv",result)

#### State_Vaccination

```State_Vaccinations(_iso_code_*, _date_*, _states_, total_vaccinations, total_distributed, people_vaccinated, people_fully_vaccinated, daily_vaccination_raw, daily_vaccinations, share_doses_used, total_boosters, population)```

In [ ]:
model_vac = read_csv_without_header("../model/Vaccination.csv")

iso_code ='USA'
fk = [data[1] for data in model_vac if data[0] == iso_code]

raw_data_states = read_csv("../covid-19-data/us_state_vaccinations.csv")

result = []
for data in raw_data_states:
    if data['date'] in fk:
        population = 0
        if data['total_vaccinations_per_hundred'].replace('.','',1).isdigit() and float(data['total_vaccinations_per_hundred']) != 0:
            population = (float(data['total_vaccinations'])/float(data['total_vaccinations_per_hundred']))*100
            
        result.append([iso_code,data['date'],data['location'],data['total_vaccinations'],data['total_distributed'],data['people_vaccinated'],data['people_fully_vaccinated'],data['daily_vaccinations_raw'],data['daily_vaccinations'],data['share_doses_used'],data['total_boosters'],population])
    else:
        print(f"{data['date']} is not in vaccination table for {iso_code}")

create_csv("../model/State_Vaccination.csv",result)

#### Vaccination_Age Schema 
```Vaccination_age(iso_code*, date*, age_group*, people_vaccinated_per_hundred, people_fully_vaccinated_per_hundred, people_with_booster_per_hundred)```

In [ ]:
fk = [f"{data[0]}{data[1]}" for data in model_vac]

raw_data_age_grp = read_csv("../covid-19-data/vaccinations-by-age-group.csv")

result = []
for data in raw_data_age_grp:
    if f"{iso_map[data['location']]}{data['date']}" in fk:
        result.append([iso_map[data['location']],data['date'],data['age_group'],data['people_vaccinated_per_hundred'],data["people_fully_vaccinated_per_hundred"],data["people_with_booster_per_hundred"]])
    else:
        print("Not in parent vaccination table",data['location'],data['date'])

create_csv("../model/Vaccination_age.csv",result)

On the data manupulation, the csv for respective schema is build and it can be imported to it's respective table in sqllite.